# Dataset

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%load_ext autoreload
%autoreload 2

In [3]:
required_libs = [ ("numpy", "numpy"),
                 ("pandas", "pandas"),
                 ("seaborn", "seaborn"),
                 ("matplotlib", "matplotlib"),
                 ("sklearn", "sklearn"),
                 ("colorama", "colorama"),
                 ("emoji", "emoji")
                ]
                 #("pycaret", "pycaret")

In [4]:
def is_lib_exists(name):
    import importlib
    lib = importlib.util.find_spec(name)
    return lib is not None

In [5]:
for (clz,lib) in required_libs:
    if not is_lib_exists(clz):
        print(f"Installing {lib}")
        !pip install {lib}
    else:
        print(f"{lib} exists")

numpy exists
pandas exists
seaborn exists
matplotlib exists
sklearn exists
colorama exists
emoji exists


In [6]:
COLAB = True

DATASET_NAME = '4D.zip'

FEATURE_DATASET_PREFIX = 'feature_matrix_d2_v3'

In [7]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import calendar
import traceback
import catboost
import gc
from pathlib import Path
from dateutil.relativedelta import *
from datetime import *
from catboost import *
from catboost import datasets
from catboost import CatBoostClassifier
from scipy import stats
from scipy.stats.stats import pearsonr
np.set_printoptions(precision=4)
pd.options.display.max_columns = None

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [8]:
%matplotlib inline
%aimport

Modules to reload:
all-except-skipped

Modules to skip:



In [9]:
from IPython.display import display

In [10]:
# check catboost version
print(catboost.__version__)
!python --version

0.24.1
Python 3.6.9


In [11]:
# colab setup
if COLAB:
  !rm -rf dl-projects
  !git clone https://github.com/mengwangk/dl-projects
  
  !cp dl-projects/utils* .
  !cp dl-projects/preprocess* .
  !cp dl-projects/plot* .
  
  from google.colab import drive
  drive.mount('/content/gdrive')
  GDRIVE_DATASET_FOLDER = Path('gdrive/My Drive/datasets/')
  DATASET_PATH = GDRIVE_DATASET_FOLDER
  ORIGIN_DATASET_PATH = Path('dl-projects/datasets')
  !ls -l gdrive/"My Drive"/datasets/ --block-size=M

DATASET = DATASET_PATH/f"{FEATURE_DATASET_PREFIX}.ft"
ORIGIN_DATASET = ORIGIN_DATASET_PATH/DATASET_NAME

Cloning into 'dl-projects'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 2228 (delta 9), reused 6 (delta 2), pack-reused 2210
Receiving objects: 100% (2228/2228), 80.11 MiB | 11.29 MiB/s, done.
Resolving deltas: 100% (1388/1388), done.
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
total 4948M
-rw------- 1 root root   17M Jul  6 14:06 feature_matrix_2020_apr.ft
-rw------- 1 root root   17M Jul  6 14:06 feature_matrix_2020_apr_orig.pkl
-rw------- 1 root root   17M Mar  1  2020 feature_matrix_2020_feb.ft
-rw------- 1 root root   17M Mar  1  2020 feature_matrix_2020_jan.ft
-rw------- 1 root root   17M Jul  7 13:52 feature_matrix_2020_jul.ft
-rw------- 1 root root   17M Jul  7 13:52 feature_matrix_2020_jul_orig.pkl
-rw------- 1 root root   17M Jul  5 07:30 feature_matrix_2020_jun.ft
-rw------- 1 root roo

## EDA

In [12]:
from preprocess import *
from utils import feature_selection, plot_feature_importances
from plot import plot_correlation_matrix, plot_labeled_scatter

In [13]:
data = pd.read_feather(DATASET)
origin_data = format_tabular(ORIGIN_DATASET)

In [14]:
jan_2020 = pd.read_feather(DATASET_PATH/f"feature_matrix_2020_jan.ft")
feb_2020 = pd.read_feather(DATASET_PATH/f"feature_matrix_2020_feb.ft")
mar_2020 = pd.read_feather(DATASET_PATH/f"feature_matrix_2020_mar.ft")
apr_2020 = pd.read_feather(DATASET_PATH/f"feature_matrix_2020_apr.ft")
may_2020 = pd.read_feather(DATASET_PATH/f"feature_matrix_2020_may.ft")
jun_2020 = pd.read_feather(DATASET_PATH/f"feature_matrix_2020_jun.ft")
jul_2020 = pd.read_feather(DATASET_PATH/f"feature_matrix_2020_jul.ft")

In [15]:
new_data = data.append(jan_2020[data.columns],ignore_index=True)
new_data = new_data.append(feb_2020[data.columns],ignore_index=True)
new_data = new_data.append(mar_2020[data.columns],ignore_index=True)
new_data = new_data.append(apr_2020[data.columns],ignore_index=True)
new_data = new_data.append(may_2020[data.columns],ignore_index=True)
new_data = new_data.append(jun_2020[data.columns],ignore_index=True)
new_data = new_data.append(jul_2020[data.columns],ignore_index=True)
data.shape, new_data.shape 

((959893, 217), (1029893, 217))

In [16]:
data = new_data

In [17]:
data.isna().sum().sort_values(ascending=False)

CUM_SUM(SKEW(Results.TotalStrike))           7685
CUM_MEAN(TREND(Results.DrawNo, DrawDate))    7685
TREND(Results.CUM_SUM(DrawNo), DrawDate)     7685
TREND(Results.CUM_SUM(LuckyNo), DrawDate)    7685
CUM_SUM(SKEW(Results.LuckyNo))               7685
                                             ... 
CUM_SUM(MIN(Results.DrawNo))                    0
NUM_UNIQUE(Results.DAY(DrawDate))               0
NUM_UNIQUE(Results.MONTH(DrawDate))             0
SUM(Results.PERCENTILE(LuckyNo))                0
NumberId                                        0
Length: 217, dtype: int64

In [18]:
data.info(max_cols=500, null_counts=True)
#data.columns.tolist()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1029893 entries, 0 to 1029892
Data columns (total 217 columns):
 #   Column                                                  Non-Null Count    Dtype         
---  ------                                                  --------------    -----         
 0   NumberId                                                1029893 non-null  int64         
 1   time                                                    1029893 non-null  datetime64[ns]
 2   STD(Results.DrawNo)                                     1028948 non-null  float64       
 3   STD(Results.TotalStrike)                                1028948 non-null  float64       
 4   STD(Results.LuckyNo)                                    1028948 non-null  float64       
 5   MAX(Results.DrawNo)                                     1029893 non-null  int64         
 6   MAX(Results.TotalStrike)                                1029893 non-null  int64         
 7   MAX(Results.LuckyNo)               

In [19]:
feature_matrix = data
feature_matrix = data.fillna(0)
feature_matrix.sort_values(by=['time', 'MAX(Results.LuckyNo)'], inplace=True)
feature_matrix.head(20)

,NumberId,time,STD(Results.DrawNo),STD(Results.TotalStrike),STD(Results.LuckyNo),MAX(Results.DrawNo),MAX(Results.TotalStrike),MAX(Results.LuckyNo),MIN(Results.DrawNo),MIN(Results.TotalStrike),MIN(Results.LuckyNo),MEAN(Results.DrawNo),MEAN(Results.TotalStrike),MEAN(Results.LuckyNo),SKEW(Results.DrawNo),SKEW(Results.TotalStrike),SKEW(Results.LuckyNo),LAST(Results.results_index),LAST(Results.DrawNo),LAST(Results.TotalStrike),LAST(Results.LuckyNo),AVG_TIME_BETWEEN(Results.DrawDate),COUNT(Results),SUM(Results.DrawNo),SUM(Results.TotalStrike),SUM(Results.LuckyNo),"TREND(Results.TotalStrike, DrawDate)","TREND(Results.LuckyNo, DrawDate)","TREND(Results.DrawNo, DrawDate)",NUM_UNIQUE(Results.PrizeType),MONTH(first_Results_time),DAY(first_Results_time),TIME_SINCE(first_Results_time),TIME_SINCE_PREVIOUS(first_Results_time),STD(Results.TIME_SINCE(DrawDate)),STD(Results.CUM_MEAN(DrawNo)),STD(Results.CUM_SUM(LuckyNo)),STD(Results.PERCENTILE(DrawNo)),STD(Results.TIME_SINCE_PREVIOUS(DrawDate)),STD(Results.CUM_SUM(TotalStrike)),STD(Results.PERCENTILE(TotalStrike)),STD(Results.CUM_SUM(DrawNo)),STD(Results.CUM_MEAN(LuckyNo)),STD(Results.CUM_MEAN(TotalStrike)),STD(Results.PERCENTILE(LuckyNo)),MAX(Results.TIME_SINCE(DrawDate)),MAX(Results.CUM_MEAN(DrawNo)),MAX(Results.CUM_SUM(LuckyNo)),MAX(Results.PERCENTILE(DrawNo)),MAX(Results.TIME_SINCE_PREVIOUS(DrawDate)),MAX(Results.CUM_SUM(TotalStrike)),MAX(Results.PERCENTILE(TotalStrike)),MAX(Results.CUM_SUM(DrawNo)),MAX(Results.CUM_MEAN(LuckyNo)),MAX(Results.CUM_MEAN(TotalStrike)),MAX(Results.PERCENTILE(LuckyNo)),MIN(Results.TIME_SINCE(DrawDate)),MIN(Results.CUM_MEAN(DrawNo)),MIN(Results.CUM_SUM(LuckyNo)),MIN(Results.PERCENTILE(DrawNo)),MIN(Results.TIME_SINCE_PREVIOUS(DrawDate)),MIN(Results.CUM_SUM(TotalStrike)),MIN(Results.PERCENTILE(TotalStrike)),MIN(Results.CUM_SUM(DrawNo)),MIN(Results.CUM_MEAN(LuckyNo)),MIN(Results.CUM_MEAN(TotalStrike)),MIN(Results.PERCENTILE(LuckyNo)),MODE(Results.MONTH(DrawDate)),MODE(Results.DAY(DrawDate)),MEAN(Results.TIME_SINCE(DrawDate)),MEAN(Results.CUM_MEAN(DrawNo)),MEAN(Results.CUM_SUM(LuckyNo)),MEAN(Results.PERCENTILE(DrawNo)),MEAN(Results.TIME_SINCE_PREVIOUS(DrawDate)),MEAN(Results.CUM_SUM(TotalStrike)),MEAN(Results.PERCENTILE(TotalStrike)),MEAN(Results.CUM_SUM(DrawNo)),MEAN(Results.CUM_MEAN(LuckyNo)),MEAN(Results.CUM_MEAN(TotalStrike)),MEAN(Results.PERCENTILE(LuckyNo)),SKEW(Results.TIME_SINCE(DrawDate)),SKEW(Results.CUM_MEAN(DrawNo)),SKEW(Results.CUM_SUM(LuckyNo)),SKEW(Results.PERCENTILE(DrawNo)),SKEW(Results.TIME_SINCE_PREVIOUS(DrawDate)),SKEW(Results.CUM_SUM(TotalStrike)),SKEW(Results.PERCENTILE(TotalStrike)),SKEW(Results.CUM_SUM(DrawNo)),SKEW(Results.CUM_MEAN(LuckyNo)),SKEW(Results.CUM_MEAN(TotalStrike)),SKEW(Results.PERCENTILE(LuckyNo)),LAST(Results.TIME_SINCE(DrawDate)),LAST(Results.CUM_MEAN(DrawNo)),LAST(Results.CUM_SUM(LuckyNo)),LAST(Results.PERCENTILE(DrawNo)),LAST(Results.DAY(DrawDate)),LAST(Results.TIME_SINCE_PREVIOUS(DrawDate)),LAST(Results.CUM_SUM(TotalStrike)),LAST(Results.MONTH(DrawDate)),LAST(Results.CUM_SUM(DrawNo)),LAST(Results.CUM_MEAN(LuckyNo)),LAST(Results.PERCENTILE(TotalStrike)),LAST(Results.CUM_MEAN(TotalStrike)),LAST(Results.PERCENTILE(LuckyNo)),SUM(Results.TIME_SINCE(DrawDate)),SUM(Results.CUM_MEAN(DrawNo)),SUM(Results.CUM_SUM(LuckyNo)),SUM(Results.PERCENTILE(DrawNo)),SUM(Results.TIME_SINCE_PREVIOUS(DrawDate)),SUM(Results.CUM_SUM(TotalStrike)),SUM(Results.PERCENTILE(TotalStrike)),SUM(Results.CUM_SUM(DrawNo)),SUM(Results.CUM_MEAN(LuckyNo)),SUM(Results.CUM_MEAN(TotalStrike)),SUM(Results.PERCENTILE(LuckyNo)),"TREND(Results.CUM_MEAN(LuckyNo), DrawDate)","TREND(Results.PERCENTILE(LuckyNo), DrawDate)","TREND(Results.PERCENTILE(DrawNo), DrawDate)","TREND(Results.CUM_MEAN(DrawNo), DrawDate)","TREND(Results.TIME_SINCE(DrawDate), DrawDate)","TREND(Results.CUM_SUM(TotalStrike), DrawDate)","TREND(Results.PERCENTILE(TotalStrike), DrawDate)","TREND(Results.TIME_SINCE_PREVIOUS(DrawDate), DrawDate)","TREND(Results.CUM_MEAN(TotalStrike), DrawDate)","TREND(Results.CU

In [20]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif, chi2, f_regression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold

other_features = ['Label', 'NumberId', 'time', 'TotalStrike',  'month', 'year']
#feature_matrix = feature_selection(data)

def select_features(df, func=f_regression):
  X = df.drop(columns=other_features)
  y = df.Label
  fs = SelectKBest(score_func=func, k=50)
  X_selected = fs.fit_transform(X, y)
  mask = fs.get_support()
  return X.columns[mask]

def reduce_features(df, lower_threshold=10, upper_threshold=10000):
  cols_to_drop = []
  for col in df.columns:
    if df[col].nunique() > upper_threshold or df[col].nunique() < lower_threshold:
      cols_to_drop.append(col)
  print(cols_to_drop)
  return cols_to_drop, df.drop(columns=cols_to_drop)

def select_by_variance(df, threshold=0):
  """Select features by variance"""
  filter = VarianceThreshold(threshold=threshold)
  filter.fit(df)
  cols_to_drop = [column for column in df.columns
                  if column not in df.columns[filter.get_support()]]
  print(f"No of columns to drop - {len(cols_to_drop)}")
  print(cols_to_drop)
  return cols_to_drop

def select_duplicates(df):
  df_transposed = df.head(20000).T
  print(df_transposed.duplicated().sum())
  unique_features = df_transposed.drop_duplicates(keep='first').T
  print(unique_features.shape)
  duplicated_features = [dup_col for dup_col in df.columns if dup_col not in unique_features.columns]
  return duplicated_features

def select_correlated(df):
  correlated_features = set()
  correlation_matrix = df.corr()
  for i in range(len(correlation_matrix .columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.8:
            colname = correlation_matrix.columns[i]
            correlated_features.add(colname)
  print(len(correlated_features))
  print(correlated_features)
  return correlated_features


In [21]:
selected_features = feature_matrix.columns.to_list()
display(len(selected_features), selected_features)

217

['NumberId',
 'time',
 'STD(Results.DrawNo)',
 'STD(Results.TotalStrike)',
 'STD(Results.LuckyNo)',
 'MAX(Results.DrawNo)',
 'MAX(Results.TotalStrike)',
 'MAX(Results.LuckyNo)',
 'MIN(Results.DrawNo)',
 'MIN(Results.TotalStrike)',
 'MIN(Results.LuckyNo)',
 'MEAN(Results.DrawNo)',
 'MEAN(Results.TotalStrike)',
 'MEAN(Results.LuckyNo)',
 'SKEW(Results.DrawNo)',
 'SKEW(Results.TotalStrike)',
 'SKEW(Results.LuckyNo)',
 'LAST(Results.results_index)',
 'LAST(Results.DrawNo)',
 'LAST(Results.TotalStrike)',
 'LAST(Results.LuckyNo)',
 'AVG_TIME_BETWEEN(Results.DrawDate)',
 'COUNT(Results)',
 'SUM(Results.DrawNo)',
 'SUM(Results.TotalStrike)',
 'SUM(Results.LuckyNo)',
 'TREND(Results.TotalStrike, DrawDate)',
 'TREND(Results.LuckyNo, DrawDate)',
 'TREND(Results.DrawNo, DrawDate)',
 'NUM_UNIQUE(Results.PrizeType)',
 'MONTH(first_Results_time)',
 'DAY(first_Results_time)',
 'TIME_SINCE(first_Results_time)',
 'TIME_SINCE_PREVIOUS(first_Results_time)',
 'STD(Results.TIME_SINCE(DrawDate))',
 'STD(Resu

In [22]:
df_selected_features = feature_matrix[selected_features]

In [23]:
def split_data(df, dt):
  y_train = df.loc[df['time'] < dt, 'Label']
  y_validation = df.loc[df['time'] == dt, 'Label']

  X_train = df[df['time'] < dt].drop(columns = ['NumberId', 'time', 'Label', 'TotalStrike', 'year'])
  X_validation = df[df['time'] == dt].drop(columns = ['NumberId', 'time', 'Label', 'TotalStrike', 'year'])

  # Reduce by standard deviation
  #X_train = X_train.loc[:, X_train.std() > .0]
  #X_validation = X_validation[X_train.columns]

  return X_train, X_validation, y_train, y_validation
 

In [24]:
TARGET_MONTH = datetime(2020,1,1)

X_train, X_validation, y_train, y_validation = split_data(df_selected_features, TARGET_MONTH)
X_train.shape, X_validation.shape, y_train.shape, y_validation.shape

((959893, 212), (10000, 212), (959893,), (10000,))

In [25]:
cols_to_drop = select_by_variance(X_train,0.0)
X_train = X_train.drop(columns=cols_to_drop)
X_validation = X_validation[X_train.columns]
print(X_train.shape, X_validation.shape)

cols_to_drop = select_by_variance(X_train,0.01)
X_train = X_train.drop(columns=cols_to_drop)
X_validation = X_validation[X_train.columns]
print(X_train.shape, X_validation.shape)

cols_to_drop = select_duplicates(X_train)
X_train = X_train.drop(columns=cols_to_drop)
X_validation = X_validation[X_train.columns]
print(X_train.shape, X_validation.shape)

cols_to_drop = select_correlated(X_train)
X_train = X_train.drop(columns=cols_to_drop)
X_validation = X_validation[X_train.columns]
print(X_train.shape, X_validation.shape)

No of columns to drop - 41
['STD(Results.TotalStrike)', 'STD(Results.LuckyNo)', 'MAX(Results.TotalStrike)', 'MIN(Results.TotalStrike)', 'MEAN(Results.TotalStrike)', 'SKEW(Results.TotalStrike)', 'SKEW(Results.LuckyNo)', 'LAST(Results.TotalStrike)', 'TREND(Results.TotalStrike, DrawDate)', 'TREND(Results.LuckyNo, DrawDate)', 'NUM_UNIQUE(Results.PrizeType)', 'STD(Results.PERCENTILE(TotalStrike))', 'STD(Results.CUM_MEAN(TotalStrike))', 'STD(Results.PERCENTILE(LuckyNo))', 'MAX(Results.CUM_MEAN(TotalStrike))', 'MIN(Results.CUM_MEAN(TotalStrike))', 'MEAN(Results.CUM_MEAN(TotalStrike))', 'SKEW(Results.PERCENTILE(TotalStrike))', 'SKEW(Results.CUM_MEAN(TotalStrike))', 'SKEW(Results.PERCENTILE(LuckyNo))', 'LAST(Results.CUM_MEAN(TotalStrike))', 'TREND(Results.CUM_MEAN(TotalStrike), DrawDate)', 'CUM_SUM(SKEW(Results.LuckyNo))', 'CUM_SUM(STD(Results.LuckyNo))', 'CUM_SUM(TREND(Results.TotalStrike, DrawDate))', 'CUM_SUM(TREND(Results.LuckyNo, DrawDate))', 'CUM_SUM(STD(Results.TotalStrike))', 'CUM_SUM(S

In [26]:
from pycaret.utils import version
print(version())

2.1.1


In [27]:
y_train

7020      0
3348      0
9337      0
6489      0
7612      0
         ..
951182    0
954761    0
950811    0
955548    0
956511    0
Name: Label, Length: 959893, dtype: int64

In [28]:
train_label = pd.DataFrame({"Label": y_train })
train_label.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 959893 entries, 7020 to 956511
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   Label   959893 non-null  int64
dtypes: int64(1)
memory usage: 14.6 MB


In [29]:
train_data = X_train
train_data['Label'] = train_label
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 959893 entries, 7020 to 956511
Data columns (total 33 columns):
 #   Column                                         Non-Null Count   Dtype  
---  ------                                         --------------   -----  
 0   STD(Results.DrawNo)                            959893 non-null  float64
 1   MAX(Results.DrawNo)                            959893 non-null  int64  
 2   MAX(Results.LuckyNo)                           959893 non-null  int64  
 3   MIN(Results.DrawNo)                            959893 non-null  int64  
 4   MEAN(Results.DrawNo)                           959893 non-null  float64
 5   SKEW(Results.DrawNo)                           959893 non-null  float64
 6   AVG_TIME_BETWEEN(Results.DrawDate)             959893 non-null  float64
 7   COUNT(Results)                                 959893 non-null  int64  
 8   MONTH(first_Results_time)                      959893 non-null  int64  
 9   DAY(first_Results_time)           

In [30]:
train_data.isna().sum().sort_values(ascending=False)
train_data.head(10)

Label                                            0
MAX(Results.CUM_MEAN(LuckyNo))                   0
MAX(Results.DrawNo)                              0
MAX(Results.LuckyNo)                             0
MIN(Results.DrawNo)                              0
MEAN(Results.DrawNo)                             0
SKEW(Results.DrawNo)                             0
AVG_TIME_BETWEEN(Results.DrawDate)               0
COUNT(Results)                                   0
MONTH(first_Results_time)                        0
DAY(first_Results_time)                          0
TIME_SINCE(first_Results_time)                   0
TIME_SINCE_PREVIOUS(first_Results_time)          0
STD(Results.TIME_SINCE_PREVIOUS(DrawDate))       0
STD(Results.CUM_MEAN(LuckyNo))                   0
MAX(Results.PERCENTILE(DrawNo))                  0
MIN(Results.TIME_SINCE_PREVIOUS(DrawDate))       0
month                                            0
MODE(Results.MONTH(DrawDate))                    0
MODE(Results.DAY(DrawDate))    

,STD(Results.DrawNo),MAX(Results.DrawNo),MAX(Results.LuckyNo),MIN(Results.DrawNo),MEAN(Results.DrawNo),SKEW(Results.DrawNo),AVG_TIME_BETWEEN(Results.DrawDate),COUNT(Results),MONTH(first_Results_time),DAY(first_Results_time),TIME_SINCE(first_Results_time),TIME_SINCE_PREVIOUS(first_Results_time),STD(Results.TIME_SINCE_PREVIOUS(DrawDate)),STD(Results.CUM_MEAN(LuckyNo)),MAX(Results.PERCENTILE(DrawNo)),MAX(Results.CUM_MEAN(LuckyNo)),MIN(Results.TIME_SINCE_PREVIOUS(DrawDate)),MODE(Results.MONTH(DrawDate)),MODE(Results.DAY(DrawDate)),MEAN(Results.TIME_SINCE(DrawDate)),SKEW(Results.CUM_MEAN(LuckyNo)),LAST(Results.DAY(DrawDate)),LAST(Results.TIME_SINCE_PREVIOUS(DrawDate)),LAST(Results.MONTH(DrawDate)),LAST(Results.CUM_MEAN(LuckyNo)),"TREND(Results.TIME_SINCE(DrawDate), DrawDate)","TREND(Results.CUM_SUM(DrawNo), DrawDate)",CUM_SUM(SKEW(Results.DrawNo)),CUM_MEAN(MEAN(Results.DrawNo)),CUM_MEAN(AVG_TIME_BETWEEN(Results.DrawDate)),"PERCENTILE(TREND(Results.DrawNo, DrawDate))",month,Label
7020,74211.178590,285107,0,91595,179383.833333,0.532566,74753280.0,6,8,2,518054340.0,0.0,105817.956888,7.801564,0.756977,4992.899701,0.0,6,1,3.461327e+08,-0.258060,6,0.0,6,4978.421900,-60.0,1376.502390,3.200162,190419.060886,8.370921e+07,0.458043,1,0
3348,96697.500291,363311,1,56893,174578.000000,0.722334,73429200.0,9,5,16,587865540.0,0.0,0.000000,6.762139,0.999232,4994.754962,0.0,5,2,3.566879e+08,1.341296,27,0.0,12,4980.496859,-60.0,1437.356485,-19.278271,183997.469377,8.419657e+07,0.463979,1,0
9337,49525.697299,265606,2,157599,224728.250000,-1.079713,68659200.0,4,9,29,386812740.0,259200.0,0.000000,8.221560,0.696568,4992.970932,0.0,4,4,2.579903e+08,1.804340,9,0.0,4,4975.946856,-60.0,1525.495147,-13.449076,197805.884268,8.327298e+07,0.772820,1,0
6489,93988.112957,347611,3,84395,176371.857143,1.067773,84009600.0,7,2,15,532569540.0,259200.0,97968.391404,6.044692,0.950595,4993.958509,0.0,2,15,3.529810e+08,0.840577,5,0.0,2,4979.532543,-60.0,1522.690584,19.160317,188983.766029,8.377023e+07,0.647462,1,0
7612,78119.505233,335810,4,100696,193401.444444,0.882965,56214000.0,9,3,2,499651140.0,0.0,0.000000,6.696385,0.914040,4996.050317,0.0,3,2,3.192863e+08,0.267884,2,0.0,6,4984.836894,-60.0,1587.798284,15.227023,191940.396526,8.360398e+07,0.780188,1,0
7416,97078.301227,360211,5,97495,268439.000000,-1.178833,100068480.0,6,12,17,506217540.0,86400.0,0.000000,2.753798,0.989629,4984.578524,0.0,5,15,1.774367e+08,0.057116,25,0.0,10,4980.341400,-60.0,1663.303668,12.145379,191453.678754,8.361350e+07,0.839439,1,0
8842,82118.479244,320409,6,131998,248964.600000,-0.850303,89877600.0,5,3,1,436665540.0,0.0,0.000000,6.191901,0.866333,4993.687247,0.0,10,1,2.126303e+08,1.016375,22,0.0,7,4984.931765,-60.0,1650.055247,7.554068,195706.025362,8.355504e+07,0.822247,1,0
7129,87999.273269,316509,7,93195,189567.666667,0.654102,86071680.0,6,9,9,514771140.0,0.0,0.000000,8.750099,0.854251,4997.520222,0.0,11,29,3.269519e+08,-0.250755,29,0.0,4,4985.332792,-60.0,1498.768299,15.476359,190657.476129,8.369209e+07,0.610827,1,0
2764,104484.902534,333810,8,53193,199387.428571,-0.091209,90345600.0,7,2,18,595382340.0,0.0,0.000000,9.155609,0.907844,4993.366996,0.0,2,1,3.092379e+08,-0.174664,24,0.0,4,4984.657205,-60.0,1398.752159,-28.842804,183237.366556,8.430943e+07,0.428162,1,0
6056,97797.735084,348611,9,79494,229070.333333,-0.560249,64443600.0,9,10,23,542505540.0,0.0,86400.000000,6.060747,0.953693,4993.367172,0.0,2,23,2.522783e+08,0.823846,23,0.0,2,4980.587875,-60.0,1488.656316,10.936651,188107.407539,8.397462e+07,0.590463,1,0


In [31]:
from pycaret.classification import *
clf1 = setup(train_data, target = 'Label', session_id=123, log_experiment=True, experiment_name='lotto')

Setup Succesfully Completed!


,Description,Value
0,session_id,123
1,Target Type,Binary
2,Label Encoded,"0: 0, 1: 1"
3,Original Data,"(959893, 33)"
4,Missing Values,False
5,Numeric Features,27
6,Categorical Features,5
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [ ]:
best_model = compare_models()

IntProgress(value=0, description='Processing: ', max=176)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,13:15:55
Status,. . . . . . . . . . . . . . . . . .,Fitting Fold 2 of 10
Estimator,. . . . . . . . . . . . . . . . . .,Logistic Regression
ETC,. . . . . . . . . . . . . . . . . .,1.88 Minutes Remaining


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
